In [ ]:
import warnings
warnings.filterwarnings('ignore', 'numpy.dtype size changed')
warnings.filterwarnings( 'ignore', category=FutureWarning)

from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from pyproj import Proj
from scipy.ndimage import gaussian_filter
import xarray as xr

In [ ]:
date = datetime(2019, 3, 14, 12)

# Current Data
# ds = xr.open_dataset('https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/GFS/Global_0p5deg_ana/TP')

# UCAR RDA Archive GFS Files 1 deg output
# ds = xr.open_dataset('https://rda.ucar.edu/thredds/dodsC/aggregations/g/ds083.2/2/TP')

# UCAR RDA Archive - Global 0.25 deg Data
ds = xr.open_dataset('https://rda.ucar.edu/thredds/dodsC/files/g/ds083.3/'
                     f'{date:%Y}/{date:%Y%m}/gdas1.fnl0p25.{date:%Y%m%d%H}.f00.grib2')

# Local File
#ds = xr.open_dataset('groundhogs_day_blizzard/GFS_{0:%Y%m%d}_{0:%H}00.nc'.format(date))

lats = ds.lat.data
lons = cutil.add_cyclic_point(ds.lon.data)

# Set subset slice for the geographic extent of data to limit download
lon_slice = slice(400,701)
lat_slice = slice(10,160)

lat1d = lats[lat_slice]
lon1d = lons[lon_slice]
lons, lats = np.meshgrid(lon1d, lat1d)

hght_dim_name = ds.Geopotential_height_isobaric.dims[1]
rhum_dim_name = ds.Relative_humidity_isobaric.dims[1]
uwnd_dim_name = ds['u-component_of_wind_isobaric'].dims[1]

ip100_hght = list(ds[hght_dim_name].values).index(10000)-1
ip100_uwnd = list(ds[uwnd_dim_name].values).index(10000)-1
ip100_rhum = list(ds[rhum_dim_name].values).index(10000)-1

lev = ds[hght_dim_name].values[ip100_hght:] * units.Pa

uwnd = ds['u-component_of_wind_isobaric'].data[0, ip100_uwnd:, lat_slice, lon_slice] * (units.meter/units.seconds)
vwnd = ds['v-component_of_wind_isobaric'].data[0, ip100_uwnd:, lat_slice, lon_slice] * (units.meter/units.seconds)
tmpk = ds['Temperature_isobaric'].data[0, ip100_hght:, lat_slice, lon_slice] * units.K
relh = ds['Relative_humidity_isobaric'].data[0, ip100_rhum:, lat_slice, lon_slice] * units.percent

vtime = datetime.strptime(str(ds.Geopotential_height_isobaric.time.data[0].astype('datetime64[ms]')),
                          '%Y-%m-%dT%H:%M:%S.%f')

mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
datacrs = ccrs.PlateCarree()

# Transform Coordinates ahead of time
tlatlons = mapcrs.transform_points(ccrs.PlateCarree(), lons, lats)
clons = tlatlons[:,:,0]
clats = tlatlons[:,:,1]

isentlevs = list(range(285, 331, 3)) * units.kelvin
wind_slice = (slice(None, None, 5), slice(None, None, 5))

isent_anal = mpcalc.isentropic_interpolation(isentlevs,
                                             lev,
                                             tmpk,
                                             relh,
                                             uwnd,
                                             vwnd,
                                             temperature_out=True)

def plot_isentropic(level):
    print("Creating the {} K Level Maps".format(level))
    ilev = list(isentlevs.m).index(level)

    isentprs = mpcalc.smooth_n_point(isent_anal[0][ilev,:,:], 9, 5)
    isenttmp = mpcalc.smooth_n_point(isent_anal[1][ilev,:,:], 9, 5)
    isentrelh = mpcalc.smooth_n_point(isent_anal[2][ilev,:,:], 9, 5)
    isentu = mpcalc.smooth_n_point(isent_anal[3].to('kt')[ilev,:,:], 9, 5)
    isentv = mpcalc.smooth_n_point(isent_anal[4].to('kt')[ilev,:,:], 9, 5)


    fig = plt.figure(1, figsize=(17,15))

    # 1st panel
    ax = plt.subplot(111, projection=mapcrs)
    ax.set_extent([-130, -72, 20, 55], ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE.with_scale('50m'))
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    cf = ax.contourf(clons, clats, isentrelh, range(70,101,1), cmap=plt.cm.Greens, norm=plt.Normalize(70,108))
    plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

    cs = ax.contour(clons, clats, isentprs, range(50,1001,50), colors='black')
    plt.clabel(cs, fmt='%d')

    ax.barbs(lons[wind_slice], lats[wind_slice],
             isentu[wind_slice].m, isentv[wind_slice].m,
             transform=ccrs.PlateCarree())

    plt.title(f'{isentlevs[ilev].m} K Pressure (hPa), Relative Humidity (%), and Wind Barbs', loc='left')
    plt.title(f'Valid Time: {vtime}', loc='right')

    plt.savefig(f'Isentropic_{isentlevs[ilev].m}K_{date:%Y%m%d_%H}00.png', bbox_inches='tight', dpi=150)
    plt.show()
    plt.close()

In [ ]:
plot_isentropic(300)